In [1]:
'''this notebook is dedicated for finding appropriate parameters for a random policy
wich will then be used as a starting point in a policy gradient tuning and debug'''

'this notebook is dedicated for finding appropriate parameters for a random policy\nwich will then be used as a starting point in a policy gradient tuning and debug'

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
from imagenet_dataset import get_dataset
from retina_env import RetinaEnv, calculate_retinal_filter
from rl_networks import create_actor_model, create_critic_model, policy
from rl_core import Buffer, update_target
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from pg_core import GaussianPolicyAgent
from rl_networks import create_actor_network_test_v1

from test_envs import SimpleMountainCarContinuous 
from misc import print_progress_bar
import types
config = types.SimpleNamespace()
config.batch_size = 1
# config.margin = 20
# config.image_h = 224
# config.image_w = 224
# config.image_hm = config.image_h+2*config.margin
# config.image_wm = config.image_w+2*config.margin
# config.foveate = None
# config.do_grayscale = True
# config.history_length = 16
# config.t_ignore = 16
# config.t_max =50
# config.motion_mode = 'velocity'
# config.use_dones = False

# config.gym_mode = False
# t_vec = np.linspace(0,150,16)

# balanced_filter = calculate_retinal_filter(t_vec, R=1.0)
# config.filter = balanced_filter.reshape([1,1,-1,1])
# config.min_freq = 1
# config.max_freq = 13
# config.action_upper_bound = np.array([2.0, 2.0])
actor_lr = 1e-4
dataset_dir = '/home/bnapp/datasets/tensorflow_datasets/imagenet2012/5.0.0/'

def epsilon_scheduler(episode, floor_episode=200, epsilon_floor=0.1):
    if episode < floor_episode:
        return 1.-(1.-epsilon_floor)*episode/floor_episode
    else:
        return epsilon_floor

# dataset = get_dataset(dataset_dir,
#                                      'validation',
#                                      config.batch_size,
#                                      image_h = config.image_hm,
#                                      image_w = config.image_wm,
#                                      preprocessing='identity',
#                                      rggb_mode=False,
#                                      central_squeeze_and_pad_factor=-1)

# # env = RetinaEnv(config, image_generator=dataset)
# env = RetinaEnv(config) #freezing on first batch
# for images,_ in dataset:
#     break
    
env =  SimpleMountainCarContinuous(max_steps=2000)




# actor_model = create_actor_model(env.image_h, env.image_w,
#                                  env.spectral_density_size, env.location_history_size,
#                                  env.timestep_size, env.action_size)
actor_model = create_actor_network_test_v1((2,), 2.0)

actor_optimizer = tf.keras.optimizers.Adam(actor_lr)
actor_model.optimizer = actor_optimizer
agent = GaussianPolicyAgent(std_deviation=0.2,model=actor_model,action_dim=1)

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [3]:
sigma_vec = [0.1,0.5,1.0,2.0,4.0]

In [5]:
for sigma in sigma_vec:
# Training loop
    reward_records = []
    epsilon_records = []
    action_mean_records = []
    action_var_records = []
    action_statmean_records = []
    action_statvar_records = []
    episodes = 1000
    for ep in range(episodes):
        prev_state = env.reset()
        episodic_reward = 0
        states, actions, rewards = [], [], []

    #     epsilon = epsilon_scheduler(ep, floor_episode=1000)

        while True:

    #         deterministic_action, means,stdevs = agent.get_action(prev_state[np.newaxis,...],
    #                                                return_stats=True)
            random_action = [[sigma *np.random.normal()]]
            action = random_action 
            #hook in order to add noise if neccessary
    #         state, reward, done, info = env.step(np.reshape(action,[1]))
            state, reward, done, info = env.step(action[0][0])

    #         if env.warmup_done:
            states.append([prev_state])
            actions.append(action)
            rewards.append(reward)

            episodic_reward += reward

    #         action_mean_records.append(deterministic_action.mean(axis=0))
    #         action_var_records.append(deterministic_action.var(axis=0))
    #         action_statmean_records.append(means)
    #         action_statvar_records.append(stdevs)
            # End this episode when `done` is True
            if done:
                break

            prev_state = np.copy(state)

    #     agent.train(np.array(states), np.array(actions), np.array(rewards))
        reward_records.append(episodic_reward)
        print_progress_bar(ep, episodes)
#         print(f"Episode * {ep} * Episodic Reward is ==> {np.mean([episodic_reward])}")
    #     print(f"Episode * {ep} * exploration epsilon {epsilon} * Episodic Reward is ==> {episodic_reward.numpy().mean()}")
    #     print("action means and variances at step -10:", action_mean_records[-10],action_var_records[-10])
    #     print("action means and variances at step -5:", action_mean_records[-5],action_var_records[-5])
    #     print("action statmeans and variances at step -10:", action_statmean_records[-10][0],action_statvar_records[-10][0])
    #     print("action statmeans and variances at step -5:", action_statmean_records[-5][0],action_statvar_records[-5][0])
    reward_records = np.array(reward_records)
    done_flag = (-reward_records < env.max_steps)
    percentage_done = done_flag.mean()
    mean_step_done = -reward_records[done_flag].mean()
    print('sigma {}, percentage_done [%] {:.2f}, mean steps for dones {:.2f}'.format(sigma, percentage_done*100, mean_step_done))

sigma 0.1, percentage_done [%] 0.00, mean steps for dones nan████████████████████████████████████████-| 99.9% 
 |----------------------------------------------------------------------------------------------------| 0.8% 

/tmp/ipykernel_3296847/746933288.py:56: RuntimeWarning: Mean of empty slice.
  mean_step_done = -reward_records[done_flag].mean()
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


sigma 0.5, percentage_done [%] 0.00, mean steps for dones nan████████████████████████████████████████-| 99.9% 
sigma 1.0, percentage_done [%] 4.20, mean steps for dones 1227.33████████████████████████████████████-| 99.9% 
sigma 2.0, percentage_done [%] 71.40, mean steps for dones 956.69████████████████████████████████████-| 99.9% 
sigma 4.0, percentage_done [%] 99.70, mean steps for dones 365.48████████████████████████████████████-| 99.9% 


In [6]:
for sigma in [6., 8., 10.,15]:
# Training loop
    reward_records = []
    epsilon_records = []
    action_mean_records = []
    action_var_records = []
    action_statmean_records = []
    action_statvar_records = []
    episodes = 1000
    for ep in range(episodes):
        prev_state = env.reset()
        episodic_reward = 0
        states, actions, rewards = [], [], []

    #     epsilon = epsilon_scheduler(ep, floor_episode=1000)

        while True:

    #         deterministic_action, means,stdevs = agent.get_action(prev_state[np.newaxis,...],
    #                                                return_stats=True)
            random_action = [[sigma *np.random.normal()]]
            action = random_action 
            #hook in order to add noise if neccessary
    #         state, reward, done, info = env.step(np.reshape(action,[1]))
            state, reward, done, info = env.step(action[0][0])

    #         if env.warmup_done:
            states.append([prev_state])
            actions.append(action)
            rewards.append(reward)

            episodic_reward += reward

    #         action_mean_records.append(deterministic_action.mean(axis=0))
    #         action_var_records.append(deterministic_action.var(axis=0))
    #         action_statmean_records.append(means)
    #         action_statvar_records.append(stdevs)
            # End this episode when `done` is True
            if done:
                break

            prev_state = np.copy(state)

    #     agent.train(np.array(states), np.array(actions), np.array(rewards))
        reward_records.append(episodic_reward)
        print_progress_bar(ep, episodes)
#         print(f"Episode * {ep} * Episodic Reward is ==> {np.mean([episodic_reward])}")
    #     print(f"Episode * {ep} * exploration epsilon {epsilon} * Episodic Reward is ==> {episodic_reward.numpy().mean()}")
    #     print("action means and variances at step -10:", action_mean_records[-10],action_var_records[-10])
    #     print("action means and variances at step -5:", action_mean_records[-5],action_var_records[-5])
    #     print("action statmeans and variances at step -10:", action_statmean_records[-10][0],action_statvar_records[-10][0])
    #     print("action statmeans and variances at step -5:", action_statmean_records[-5][0],action_statvar_records[-5][0])
    reward_records = np.array(reward_records)
    done_flag = (-reward_records < env.max_steps)
    percentage_done = done_flag.mean()
    mean_step_done = -reward_records[done_flag].mean()
    print('sigma {}, percentage_done [%] {:.2f}, mean steps for dones {:.2f}'.format(sigma, percentage_done*100, mean_step_done))

sigma 6.0, percentage_done [%] 100.00, mean steps for dones 185.27███████████████████████████████████-| 99.9% 
sigma 8.0, percentage_done [%] 100.00, mean steps for dones 118.12███████████████████████████████████-| 99.9% 
sigma 10.0, percentage_done [%] 100.00, mean steps for dones 76.55███████████████████████████████████-| 99.9% 
sigma 15, percentage_done [%] 100.00, mean steps for dones 60.99█████████████████████████████████████-| 99.9% 


In [7]:
for sigma in [20.,40.,80]:
# Training loop
    reward_records = []
    epsilon_records = []
    action_mean_records = []
    action_var_records = []
    action_statmean_records = []
    action_statvar_records = []
    episodes = 1000
    for ep in range(episodes):
        prev_state = env.reset()
        episodic_reward = 0
        states, actions, rewards = [], [], []

    #     epsilon = epsilon_scheduler(ep, floor_episode=1000)

        while True:

    #         deterministic_action, means,stdevs = agent.get_action(prev_state[np.newaxis,...],
    #                                                return_stats=True)
            random_action = [[sigma *np.random.normal()]]
            action = random_action 
            #hook in order to add noise if neccessary
    #         state, reward, done, info = env.step(np.reshape(action,[1]))
            state, reward, done, info = env.step(action[0][0])

    #         if env.warmup_done:
            states.append([prev_state])
            actions.append(action)
            rewards.append(reward)

            episodic_reward += reward

    #         action_mean_records.append(deterministic_action.mean(axis=0))
    #         action_var_records.append(deterministic_action.var(axis=0))
    #         action_statmean_records.append(means)
    #         action_statvar_records.append(stdevs)
            # End this episode when `done` is True
            if done:
                break

            prev_state = np.copy(state)

    #     agent.train(np.array(states), np.array(actions), np.array(rewards))
        reward_records.append(episodic_reward)
        print_progress_bar(ep, episodes)
#         print(f"Episode * {ep} * Episodic Reward is ==> {np.mean([episodic_reward])}")
    #     print(f"Episode * {ep} * exploration epsilon {epsilon} * Episodic Reward is ==> {episodic_reward.numpy().mean()}")
    #     print("action means and variances at step -10:", action_mean_records[-10],action_var_records[-10])
    #     print("action means and variances at step -5:", action_mean_records[-5],action_var_records[-5])
    #     print("action statmeans and variances at step -10:", action_statmean_records[-10][0],action_statvar_records[-10][0])
    #     print("action statmeans and variances at step -5:", action_statmean_records[-5][0],action_statvar_records[-5][0])
    reward_records = np.array(reward_records)
    done_flag = (-reward_records < env.max_steps)
    percentage_done = done_flag.mean()
    mean_step_done = -reward_records[done_flag].mean()
    print('sigma {}, percentage_done [%] {:.2f}, mean steps for dones {:.2f}'.format(sigma, percentage_done*100, mean_step_done))

sigma 20.0, percentage_done [%] 100.00, mean steps for dones 69.53███████████████████████████████████-| 99.9% 
sigma 40.0, percentage_done [%] 100.00, mean steps for dones 113.77██████████████████████████████████-| 99.9% 
sigma 80, percentage_done [%] 99.90, mean steps for dones 198.62█████████████████████████████████████-| 99.9% 


In [ ]:
for uu in range(150):
    for zz in range(1000000):
        x = np.random.normal((50))
    print_progress_bar(uu,149)